In [4]:
using AgentBasedModels
abm = AgentBasedModels

using Random
using Distributions
using CUDA

using Makie

┌ Info: Precompiling AgentBasedModels [388cb286-f2b1-4654-a3bb-2e137a39c658]
└ @ Base loading.jl:1317
  ** incremental compilation may be fatally broken for this module **

ERROR: LoadError: LoadError: LoadError: LoadError: MethodError: no method matching var"@capture"(::LineNumberNode, ::Module, ::Expr)
Closest candidates are:
  var"@capture"(::LineNumberNode, ::Module, ::Any, ::Any) at /home/gabriel/.julia/packages/MacroTools/gME9C/src/match/macro.jl:67
Stacktrace:
 [1] (::DocStringExtensions.var"#29#30"{typeof(DocStringExtensions.template_hook)})(::LineNumberNode, ::Vararg{Any, N} where N)
   @ DocStringExtensions ~/.julia/packages/DocStringExtensions/9Ot5N/src/templates.jl:11
 [2] var"@doc"(__source__::LineNumberNode, __module__::Module, x::Vararg{Any, N} where N)
   @ Core ./boot.jl:508
 [3] include(x::String)
   @ AgentBasedModels ~/Documents/PhD/Projects/Agent Based Simulator/AgentBasedModels.jl/src/AgentBasedModels.jl:1
 [4] top-level scope
   @ ~/Documents/PhD/Projects/Agent B

LoadError: Failed to precompile AgentBasedModels [388cb286-f2b1-4654-a3bb-2e137a39c658] to /home/gabriel/.julia/compiled/v1.6/AgentBasedModels/jl_tUtBKB.

# Define the agent

In [2]:
m = @agent(saiz,
    
    #Mechanics
    
    [x,y,z,vx,vy,vz]::Local, #Variables
    [Fix,Fiy,Fiz]::Local,    #Interaction forces
    id::Identity,            #Identity
    [m,r]::Local,            #Mass and radius of the model
    [μ,b]::Global,           #Global parameters of the model
    F₀::GlobalArray,         #Matrix of interaction forces
    
    Equation = begin
        d_vx = (-b*vx/m+Fix/m)*dt
        d_vy = (-b*vy/m+Fiy/m)*dt
        d_vz = (-b*vz/m+Fiz/m)*dt
        d_x = vx*dt
        d_y = vy*dt
        d_z = vz*dt
    end,
    
    UpdateInteraction = begin
        dij = sqrt((dx_i-dx_j)^2+(dy_i-dy_j)^2+(dz_i-dz_j)^2)
        rij = r_i+r_j
        if dij < μ*rij && dij > 0
            Fix_i += F₀[id_i,id_j]*(rij/dij-1)*(μ*rij/dij-1)*(x_i-x_j)/dij
            Fiy_i += F₀[id_i,id_j]*(rij/dij-1)*(μ*rij/dij-1)*(y_i-y_j)/dij
            Fiz_i += F₀[id_i,id_j]*(rij/dij-1)*(μ*rij/dij-1)*(z_i-z_j)/dij   
        end
    end,
    
    #Biochemistry
    
    c::Local,             #Biochemical component
    [ci,ni]::Local,       #Interaction parameters for the mean
    [α,n,mm,K]::Global,
    frange::Global,       #Distance of communication
    [toff,tonn]::Global,
    
    Equation = begin
        if t < toff && t > ton
            d_c = (α*(1+c^n)^mm/((1+c^n)^mm+(1+(ci/ni)/K)^(2*mm))-c)*dt
        else
            d_c = 0*dt
        end
    end,
        
    UpdateInteraction= begin
        if dij < frange*rij #Unnecessary to compute dij and rij again, previously computed in UpdateInteraction
            ni += 1
            ci += c_j
        end 
    end,
    
    #Growth
    
    tu::Local,
    [τdiv,σdiv,σc]::Global,

    EventDivision = begin
        #Choose random direction in unit sphere
        xₐ = Normal(0,1); yₐ = Normal(0,1); zₐ = Normal(0,1)
        Tₐ = sqrt(xₐ^2+yₐ^2+zₐ^2)
        xₐ /= Tₐ;yₐ /= Tₐ;zₐ /= Tₐ    

        #Chose a random distribution
        dist = Uniform(1+σc1,1-σc)

        #Update things of first cell
        x_1 = x+r*xₐ/2; y_1 = y+r*yₐ/2; z_1 = z+r*zₐ/2
        vx_1 = 0.; vy_1 = 0.; vz_1 = 0.
        r_1 = r/2. ^(1. /3)
        m_1 = m/2
        tu_1 = t + Uniform(τdiv-σdiv,τdiv+σdiv)
        c_1 = c*dist
        
        #Update things of second cell
        x_2 = x+r*xₐ/2; y_2 = y+r*yₐ/2; z_2 = z+r*zₐ/2
        vx_2 = 0.; vy_2 = 0.; vz_2 = 0.
        r_2 = r/2. ^(1. /3)
        m_2 = m/2
        tu_2 = t + Uniform(τdiv-σdiv,τdiv+σdiv)
        c_2 = c*(2-dist)
    end,
)

LoadError: LoadError: UndefVarError: @agent not defined
in expression starting at In[2]:1

# Compile the model

In [3]:
mCompiled = compile(m,platform="cpu",save="RAM",integrator="Heun",debug=true)

LoadError: UndefVarError: compile not defined

### Test final model

In [17]:
compile!(m,platform="cpu",saveRAM=true,saveVTK=false,positionsVTK=[:x,:y,:z],debug=false)

In [18]:
com = Community(m,N=1)

##########Mechanics##############################
#Initialise global
com[:b] = 10^-6
com[:F₀] = 10^-4
com[:μ] = 2

#########Division#################################
#Initialise global
com[:τdiv] = 10
com[:σdiv] = 0.5
com[:σx] = 0.01

#########Chemical#################################
#Initialise Global
com[:ton] = 0 #Start deactivated
com[:Ncirc] = 20
com[:fmin] = 0.05
com[:fmax] = 0.95
com[:α] = 10
com[:n] = 2
com[:mm] = 2
com[:K] = 0.9
com[:frange] = 1.2

#########Local parameters and variables###########
#Initialise locals
mᵢ = 10^-6
rᵢ = 5
com[:m] = mᵢ
com[:r] = rᵢ
com[:toff] = 1 #Start activated
com[:fate] = 0 #Start neutral fate
#Initialise variables
com[:x] = 0. #They are only separed in this axis
com[:y] = 0.
com[:z] = 0.
com[:vx] = 0.
com[:vy] = 0.
com[:vz] = 0.

x₀=3.
com[:xx] = x₀
com[:tu] = 10.

10.0

In [19]:
c = m.evolve(com,dt=0.001,tMax_=60,tSaveStep_=1,nMax_=1000);

In [20]:
c[last][:xx];

## Visualization

### Plot community

In [21]:
figure = plotCommunitySpheres(c[50],[:x,:y,:z],:r,:fate)
display(figure)

GLMakie.Screen(...)

### Make video

In [22]:
using Makie
using AbstractPlotting

plotDivisionTree (generic function with 2 methods)

In [44]:
names(Dict([:a=>1]))

LoadError: MethodError: no method matching names(::Dict{Symbol,Int64})
Closest candidates are:
  names(!Matched::DataFrames.Index) at /home/gabriel/.julia/packages/DataFrames/Zx5mm/src/other/index.jl:34
  names(!Matched::Module; all, imported) at reflection.jl:98
  names(!Matched::DataFrames.SubIndex) at /home/gabriel/.julia/packages/DataFrames/Zx5mm/src/other/index.jl:425
  ...

In [41]:
figure = plotDivisionTree(c,:fate)
display(figure)

LoadError: UndefVarError: key not defined